In [1]:
import cv2
import numpy as np
import os
from tqdm import tqdm 
from ocr import page, words

In [6]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import json

output_folder = "cropped_letters"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

image_path = r"farp,small,wall_texture,product,750x1000.jpg"
image = cv2.imread(image_path)
image = page.detection(image)  # Ensure `page.detection()` is correctly defined

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 120, 250, cv2.THRESH_BINARY_INV)

contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f"Total contours detected: {len(contours)}")

bounding_boxes = []
padding = 10  

for i, contour in tqdm(enumerate(contours), total=len(contours), desc="Processing Contours", unit="contour"):
    x, y, w, h = cv2.boundingRect(contour)
    
    if w > 10 and h > 10 and cv2.contourArea(contour) > 50:  
        x_pad = max(x - padding, 0)
        y_pad = max(y - padding, 0)
        w_pad = min(x + w + padding, image.shape[1]) - x_pad
        h_pad = min(y + h + padding, image.shape[0]) - y_pad
        
        letter = image[y_pad:y_pad+h_pad, x_pad:x_pad+w_pad]
        letter_gray = cv2.cvtColor(letter, cv2.COLOR_BGR2GRAY)
        _, letter_thresh = cv2.threshold(letter_gray, 160, 255, cv2.THRESH_BINARY_INV)

        letter_alpha = cv2.merge((letter_thresh, letter_thresh, letter_thresh, letter_thresh))

        filename = os.path.join(output_folder, f'letter_{len(bounding_boxes) + 1}.png')
        cv2.imwrite(filename, letter_alpha)

        bounding_boxes.append({
            "x": x_pad, "y": y_pad, "w": w_pad, "h": h_pad,
            "filename": filename
        })

# Save bounding box data
with open(os.path.join(output_folder, "bounding_boxes.json"), "w") as f:
    json.dump(bounding_boxes, f)

print("Character images and bounding box data saved successfully!")


Total contours detected: 6


Processing Contours: 100%|██████████| 6/6 [00:00<00:00, 1004.70contour/s]

Character images and bounding box data saved successfully!


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
# model = tf.keras.models.load_model('malayalam_handwriting_model.h5')

In [5]:
def preprocessing_fun(img):
    img=np.where(img>0.5,1,0)
    return img

In [6]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_fun)

In [7]:
test_generator = test_datagen.flow_from_directory(
        'cropped_letters',
        target_size=(128,128),
        batch_size=1,
        class_mode='categorical',color_mode="grayscale"
)

Found 0 images belonging to 0 classes.


In [4]:
from tensorflow.keras.models import load_model
import pickle
from tqdm import tqdm

import glob


In [5]:
test_fold=sorted(glob.glob("cropped_letters/*"))

In [7]:
model = load_model("D:\Projects\AIML\DIGILIPI\Malayalam_detection.h5")
print("Keras model loaded successfully!")
max_length = 100 


with open("detail.pkl", 'rb') as f:
    loaded_pickle_data = pickle.load(f)

# model = model_from_json(loaded_pickle_data['model'])
# model.set_weights(loaded_pickle_data['weights'])

# tokenizer = loaded_pickle_data['tokenizer']
class_labels = loaded_pickle_data['class_labels']

print("Pickle model, tokenizer, and label encoder loaded successfully!")
from sklearn.metrics import classification_report
target_names = loaded_pickle_data['target_names']
# target_names = loaded_pickle_data['target_names']




Keras model loaded successfully!
Pickle model, tokenizer, and label encoder loaded successfully!


In [8]:
import cv2
import os
import json
import numpy as np
import glob

# Load bounding box data
with open("cropped_letters/bounding_boxes.json", "r") as f:
    bounding_boxes = json.load(f)

# Sort bounding boxes by position (top to bottom, then left to right)
bounding_boxes.sort(key=lambda b: (b["y"], b["x"]))

# Load cropped images for prediction
test_fold = sorted(glob.glob("cropped_letters/*.png"))

y_pred = []
file_list = []

for img, box in zip(test_fold, bounding_boxes):  # Ensure correct mapping
    i = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    i = cv2.resize(i, (128, 128), interpolation=cv2.INTER_LINEAR)
    
    # Prepare for model prediction
    i = np.reshape(i, (1, 128, 128, 1))

    prediction = model.predict(i, verbose=0)
    p = np.argmax(prediction[0])
    
    recognized_char = target_names[p]  # Get the predicted character
    y_pred.append((box["y"], box["x"], recognized_char))  # Store with position

# Sort predictions based on position (y first, then x)
y_pred.sort()

# Extract only characters in sorted order
recognized_text = "".join([char for _, _, char in y_pred])

# Save to text file
with open("recognized_text.txt", "w", encoding="utf-8") as f:
    f.write(recognized_text)

print("Recognized text saved to 'recognized_text.txt' ✅")


Recognized text saved to 'recognized_text.txt' ✅


In [13]:
print(target_names[26])


ഛ
